# T-method Duct Design
---

- T-method optimization algorithm for duct design, using R.J. Tsal paper as reference

In [1]:
import numpy as np
import pandas as pd


from IPython.display import display, clear_output
from ipywidgets import widgets

# custom modules 
from duct import DuctSection, DuctSystem
import data_widget

In [2]:
%load_ext autoreload
%autoreload 2

---
### Input Data

- See [Nomenclature](#nomenclature_cell) for variable definitions

---
** General Data: **

In [3]:
# air temperature (deg C)
temp = 22
# absolute roughness  (m)
e = 0.0003
# kinematic viscosity (m^2/s)
kVisc = 1.54*10**(-5)
# air density        (kg/m^3)
density = 1.2
# fan efficiency
efficiency_fan_peak = 0.85
efficiency_fan_op = 0.75
# motor efficiency
efficiency_motor = 0.8
# total system airflow  (m^3/s)
Qfan = 1.42

** Economic Data: **

In [4]:
energy_cost = 2.03      # (c/kWh)
duct_cost = 43.27       # ($/m^2)
sys_oper_time = 4400    # (h/yr)

In [5]:
data_widget.display_form()

In [27]:
def matrix_to_system(matrix):
    """
    Converts data matrix to DuctSystem instance
    """
    System = DuctSystem()
    for s in matrix:
        section = DuctSection( _id=int(s[0]), ch1=int(s[1]), ch2=int(s[2]), is_rect=s[3], is_presized=s[4], \
                          air_flow=s[5], duct_length=s[6], delta_p_z=s[7], rough_factor=s[8], \
                          req_height=s[9], req_width=s[10], req_dia=s[11], loss_coeff=s[12], \
                          f_factor=0.019)
        System.add_section( int(s[0]), section)
    System.parents = System.find_parents()

    return System

def display_data(system):

    system.data_to_csv('test')
    test_df = pd.read_csv('test.csv')

    input_data = test_df[["_id", "ch1", "ch2", "is_rect", "is_presized", "air_flow", 
                          "duct_length", "delta_p_z", "rough_factor", "req_height",
                          "req_width", "req_dia", "loss_coeff"]]

    vel_size_fric_data = test_df[["_id", "velocity", "height", "width", "dia_by_frict", 
                                  "dia_by_vel", "f_factor", "total_delta_P" ]]

    condense_data = test_df[["_id","mu", "Ks", "Kt", "T","max_delta_P"]]

    expand_data = test_df[["_id","up_press", "req_delta_P", "req_delta_P_total", "down_press", "est_dia"]]

    press_loss_data = test_df[["_id", "total_path_delta_P", "exces_path_delta_P"]]

    print("Input Data:")
    display(input_data)
    print("Velocity-Size-Friction Data:")
    display(vel_size_fric_data)
    print("Condensing Data:")
    display(condense_data)
    print("Expansion Data:")
    display(expand_data)
    print("Pressure Loss Data:")
    display(press_loss_data)

** Input Test Data from csv: **

In [47]:
test_data = pd.read_csv('test_data.csv')
test_data = np.round(test_data.drop(test_data.columns[[0]], axis=1), 3)
test_data[["$S$", "$S_1$", "$S_2$"]] = test_data[["$S$", "$S_1$", "$S_2$"]].astype('int')

In [48]:
test_system = matrix_to_system(test_data.as_matrix())

## Data Initiation

In [49]:
test_system.data_init()


In [50]:
display_data(test_system)

Input Data:


,_id,ch1,ch2,is_rect,is_presized,air_flow,duct_length,delta_p_z,rough_factor,req_height,req_width,req_dia,loss_coeff
0,1,0,0,True,False,0.70,14.00,25.0,0.0,0.254,0.0,0.00,0.80
1,2,0,0,False,False,0.22,12.00,37.5,0.0,0.000,0.0,0.00,0.65
2,3,1,2,False,True,0.92,8.00,0.0,0.0,0.000,0.0,0.33,0.18
3,4,0,0,False,False,0.50,16.00,0.0,0.0,0.000,0.0,0.00,0.65
4,5,3,4,False,False,1.42,19.81,37.5,0.0,0.000,0.0,0.00,1.50


Velocity-Size-Friction Data:


,_id,velocity,height,width,dia_by_frict,dia_by_vel,f_factor,total_delta_P
0,1,6.500000,0.254,0.423985,0.317683,0.370170,0.019,66.505860
1,2,6.500000,0.000,0.000000,0.207522,0.207522,0.019,81.829029
2,3,10.762902,0.000,0.000000,0.330000,0.330000,0.019,44.524704
3,4,6.500000,0.000,0.000000,0.312851,0.312851,0.019,41.110319
4,5,6.500000,0.000,0.000000,0.527226,0.527226,0.019,93.622536


Condensing Data:


,_id,mu,Ks,Kt,T,max_delta_P
0,1,0.606084,12.800500,12.800500,1.000000,25.000000
1,2,0.362889,5.346938,5.346938,1.000000,37.500000
2,3,0.211400,0.000000,18.126410,0.000000,82.024704
3,4,0.507353,10.586930,10.586930,1.000000,0.000000
4,5,1.167229,23.508829,59.859523,0.459075,119.524704


Expansion Data:


,_id,up_press,req_delta_P,req_delta_P_total,down_press,est_dia
0,1,191.719937,166.719937,191.719937,0.000000,0.280416
1,2,191.719937,154.219937,191.719937,0.000000,0.161790
2,3,236.244642,44.524704,44.524704,191.719937,0.330000
3,4,236.244642,236.244642,236.244642,0.000000,0.220614
4,5,404.628684,130.884042,168.384042,236.244642,0.445271


Pressure Loss Data:


,_id,total_path_delta_P,exces_path_delta_P
0,1,204.653101,199.975583
1,2,219.976270,184.652414
2,3,138.147240,NaN
3,4,134.732855,269.895829
4,5,93.622536,NaN


In [36]:
#test_system.parents

## First Iteration

In [51]:
test_system.iterate()

In [52]:
display_data(test_system)

Input Data:


,_id,ch1,ch2,is_rect,is_presized,air_flow,duct_length,delta_p_z,rough_factor,req_height,req_width,req_dia,loss_coeff
0,1,0,0,True,False,0.70,14.00,25.0,0.0,0.254,0.0,0.00,0.80
1,2,0,0,False,False,0.22,12.00,37.5,0.0,0.000,0.0,0.00,0.65
2,3,1,2,False,True,0.92,8.00,0.0,0.0,0.000,0.0,0.33,0.18
3,4,0,0,False,False,0.50,16.00,0.0,0.0,0.000,0.0,0.00,0.65
4,5,3,4,False,False,1.42,19.81,37.5,0.0,0.000,0.0,0.00,1.50


Velocity-Size-Friction Data:


,_id,velocity,height,width,dia_by_frict,dia_by_vel,f_factor,total_delta_P
0,1,11.340236,0.254,0.24302,0.248389,0.280416,0.021929,182.099797
1,2,10.707568,0.000,0.00000,0.161790,0.161790,0.024495,207.192638
2,3,10.762902,0.000,0.00000,0.330000,0.330000,0.020490,47.035363
3,4,13.088033,0.000,0.00000,0.220614,0.220614,0.022410,233.846011
4,5,9.124513,0.000,0.00000,0.445271,0.445271,0.019217,155.140026


Condensing Data:


,_id,mu,Ks,Kt,T,max_delta_P
0,1,0.570931,12.247430,12.247430,1.000000,25.000000
1,2,0.399099,5.449623,5.449623,1.000000,37.500000
2,3,0.223320,0.000000,17.676725,0.000000,84.535363
3,4,0.501952,10.564294,10.564294,1.000000,0.000000
4,5,1.048597,23.010259,58.744698,0.458068,122.035363


Expansion Data:


,_id,up_press,req_delta_P,req_delta_P_total,down_press,est_dia
0,1,189.605339,164.605339,189.605339,0.000000,0.277793
1,2,189.605339,152.105339,189.605339,0.000000,0.165353
2,3,236.640701,47.035363,47.035363,189.605339,0.330000
3,4,236.640701,236.640701,236.640701,0.000000,0.220068
4,5,402.707603,128.566902,166.066902,236.640701,0.437387


Pressure Loss Data:


,_id,total_path_delta_P,exces_path_delta_P
0,1,384.275186,18.432418
1,2,409.368027,-6.660424
2,3,202.175389,NaN
3,4,388.986037,13.721567
4,5,155.140026,NaN


## Second Iteration

In [53]:
test_system.iterate()

In [54]:
display_data(test_system)

Input Data:


,_id,ch1,ch2,is_rect,is_presized,air_flow,duct_length,delta_p_z,rough_factor,req_height,req_width,req_dia,loss_coeff
0,1,0,0,True,False,0.70,14.00,25.0,0.0,0.254,0.0,0.00,0.80
1,2,0,0,False,False,0.22,12.00,37.5,0.0,0.000,0.0,0.00,0.65
2,3,1,2,False,True,0.92,8.00,0.0,0.0,0.000,0.0,0.33,0.18
3,4,0,0,False,False,0.50,16.00,0.0,0.0,0.000,0.0,0.00,0.65
4,5,3,4,False,False,1.42,19.81,37.5,0.0,0.000,0.0,0.00,1.50


Velocity-Size-Friction Data:


,_id,velocity,height,width,dia_by_frict,dia_by_vel,f_factor,total_delta_P
0,1,11.555386,0.254,0.238495,0.246004,0.277793,0.021958,189.209042
1,2,10.251083,0.000,0.000000,0.165353,0.165353,0.024428,190.259182
2,3,10.762902,0.000,0.000000,0.330000,0.330000,0.020490,47.035363
3,4,13.152990,0.000,0.000000,0.220068,0.220068,0.022418,236.651791
4,5,9.456396,0.000,0.000000,0.437387,0.437387,0.019256,164.774943


Condensing Data:


,_id,mu,Ks,Kt,T,max_delta_P
0,1,0.569376,12.243833,12.243833,1.000000,25.000000
1,2,0.400615,5.453758,5.453758,1.000000,37.500000
2,3,0.223320,0.000000,17.677262,0.000000,84.535363
3,4,0.501726,10.563343,10.563343,1.000000,0.000000
4,5,1.037544,22.961544,58.687356,0.457632,122.035363


Expansion Data:


,_id,up_press,req_delta_P,req_delta_P_total,down_press,est_dia
0,1,189.603829,164.603829,189.603829,0.000000,0.277642
1,2,189.603829,152.103829,189.603829,0.000000,0.165478
2,3,236.639191,47.035363,47.035363,189.603829,0.330000
3,4,236.639191,236.639191,236.639191,0.000000,0.220049
4,5,402.479278,128.340087,165.840087,236.639191,0.436615


Pressure Loss Data:


,_id,total_path_delta_P,exces_path_delta_P
0,1,401.019347,1.459931
1,2,402.069487,0.409791
2,3,211.810305,NaN
3,4,401.426733,1.052545
4,5,164.774943,NaN


---
### Input Data Widget

** Initial Velocities **

<a id='nomenclature_cell'></a>

---
## Nomenclature

$Q =$ Air Flow

$L =$ Duct Length

$\Delta P_z = $ Additional Pressure Loss

$H_z =$ Required Duct Height

$W_z =$ Required Duct Width

$D_z =$ Required Duct Diameter

$C = $ Local Loss Coefficient